In [1]:
import torch
import torch.nn as nn
import json
import yaml
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from PIL import Image, ImageDraw, ImageOps
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
from torchvision.transforms import v2
import matplotlib.pyplot as plt
import os
import random
import math

Mount cloud drive with data on GC

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!ls /content/drive/MyDrive/ML_IDA24_USDCdataset

README.dataset.txt   submission.csv	   test_images	 usdc_train.json
README.roboflow.txt  test_file_names.json  train_images


Main PATHs

In [3]:
!cp -r /content/drive/MyDrive/ML_IDA24_USDCdataset /content/ML_IDA24_USDCdataset
# !cp -r /content/drive/MyDrive/ML_IDA24_USDC_YOLO/USDC /content/USDC #if cahced

In [7]:
MODEL_OUTPUTS_DIR = '/content/ML_IDA24_USDCmodelingspace/'

In [8]:
if not os.path.exists(MODEL_OUTPUTS_DIR):
  !mkdir {MODEL_OUTPUTS_DIR}

In [7]:
!ls {TRAIN_DATA_DIR}

ls: cannot access './content/ML_IDA24_USDCdataset/train_images/train_images/': No such file or directory


In [8]:
!ls {TEST_DATA_DIR}

ls: cannot access './content/ML_IDA24_USDCdataset/test_images/test_images/': No such file or directory


Preparation

In [32]:
if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 17067, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 17067 (delta 24), reused 28 (delta 12), pack-reused 17022 (from 1)
Receiving objects: 100% (17067/17067), 15.68 MiB | 16.27 MiB/s, done.
Resolving deltas: 100% (11719/11719), done.


In [33]:
!pip install -r /content/yolov5/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.3/896.3 kB 23.0 MB/s eta 0:00:00


In [7]:
if not os.path.exists('JSON2YOLO'):
    !git clone https://github.com/ultralytics/JSON2YOLO.git

Cloning into 'JSON2YOLO'...
remote: Enumerating objects: 421, done.
remote: Counting objects: 100% (285/285), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 421 (delta 243), reused 199 (delta 184), pack-reused 136 (from 1)
Receiving objects: 100% (421/421), 120.25 KiB | 1.02 MiB/s, done.
Resolving deltas: 100% (259/259), done.


In [14]:
!pip install -r /content/JSON2YOLO/requirements.txt

In [19]:
%env PYTHONPATH="$/env/python:/content/JSON2YOLO/utils.py"

env: PYTHONPATH="$/env/python:/content/JSON2YOLO/utils.py"


DataSet conversion

In [9]:
from JSON2YOLO.general_json2yolo import convert_coco_json

In [11]:
!mkdir annotation_train

mkdir: cannot create directory ‘annotation_train’: File exists


In [62]:
!mv ./ML_IDA24_USDCdataset/usdc_train.json ./annotation_train

In [12]:
convert_coco_json('./annotation_train', False, False)

[PosixPath('/content/annotation_train/usdc_train.json')]


Annotations /content/annotation_train/usdc_train.json: 100%|██████████| 10597/10597 [00:02<00:00, 3661.89it/s]


In [18]:
!mkdir -p /content/USDC/images/train_usdc

In [19]:
!mkdir -p /content/USDC/labels/train_usdc

In [20]:
!mkdir -p /content/USDC/images/test_usdc

In [21]:
!mv /content/ML_IDA24_USDCdataset/train_images/train_images/* /content/USDC/images/train_usdc
!mv /content/new_dir/labels/usdc_train/* /content/USDC/labels/train_usdc
!mv /content/ML_IDA24_USDCdataset/test_images/test_images/* /content/USDC/images/test_usdc

In [24]:
%cd /content/USDC/
!find ./images/train_usdc -name "*.jpg" -printf '%p\n' > /content/USDC/train_usdc.txt
!find ./images/test_usdc -name "*.jpg" -printf '%p\n' > /content/USDC/test_usdc.txt

/content/USDC


In [25]:
%cd /content/

/content


In [26]:
#make yaml

with open('./annotation_train/usdc_train.json', 'r') as file:
    train_data = json.load(file)
classes_dict = {}
for sample in train_data['categories']:
    classes_dict[sample['id']-1] = sample['name']
del classes_dict[-1]
classes_dict

d = {
    "path": "/content/USDC/",
    "train": "train_usdc.txt",
    "val": "train_usdc.txt",
    "test": "test_usdc.txt",
    "names": classes_dict,
}

with open("/content/USDC/data.yaml", "w") as f:
    yaml.dump(d, f, sort_keys=False)

In [29]:
#cache to cloud
!mkdir -p /content/drive/MyDrive/ML_IDA24_USDC_YOLO

!cp -r /content/USDC/ /content/drive/MyDrive/ML_IDA24_USDC_YOLO

In [31]:
!ls /content/drive/MyDrive/ML_IDA24_USDC_YOLO/USDC

data.yaml  images  labels  test_usdc.txt  train_usdc.txt


In [34]:
EPOCHS = 1

In [18]:
VANILLA_DIRS = "vanilla_v5s"

In [19]:
if not os.path.exists(MODEL_OUTPUTS_DIR):
  !mkdir -p {MODEL_OUTPUTS_DIR}/{VANILLA_DIRS}

In [20]:
LC_DIRS = "lc_modified_v5s"

In [21]:
if not os.path.exists(MODEL_OUTPUTS_DIR):
  !mkdir -p {MODEL_OUTPUTS_DIR}/{LC_DIRS}

In [22]:
!nvidia-smi

Wed Nov 27 21:12:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Make data yaml in YOLOv5 repo format

Vanilla YOLOv5S tuning for custom dataset

In [39]:
!python /content/yolov5/train.py --data /content/USDC/data.yaml --weights yolov5s.pt \
    --img 512 --epochs {EPOCHS} --batch-size 1 --name {MODEL_OUTPUTS_DIR}/train

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-11-28 00:17:40.074384: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-28 00:17:40.111034: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-28 00:17:40.121409: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visua

In [ ]:
3

YOLOv5S tuning for custom dataset with LCN

Utilitary functoins